In [5]:
import psycopg2
from credentials import databaseKeys

In [8]:
connection = psycopg2.connect(
    host = 'db-cc.co4twflu4ebv.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = databaseKeys.username,
    password = databaseKeys.password,
    database='lion_leftovers'
    )
cursor=connection.cursor()

In [6]:
#create inventory table 

cursor.execute("""
CREATE TABLE Inventory(
    InventoryID SERIAL PRIMARY KEY,
    DiningHallID VARCHAR(255),
    FoodItem VARCHAR(255),
    Quantity INT,
    Price DECIMAL,
    ExpirationTime TIMESTAMP,
    LastUpdated TIMESTAMP
)""")

In [ ]:
#create order table
cursor.execute("""
CREATE TABLE Orders(
    OrderID SERIAL PRIMARY KEY,
    StudentUNI VARCHAR(10),
    InventoryID INT,
    OrderQuantity INT,
    TotalPrice DECIMAL,
    OrderTime TIMESTAMP,
    PaymentStatus VARCHAR(50),
    ReceiptNumber VARCHAR(255),
    FOREIGN KEY (InventoryID) REFERENCES Inventory(InventoryID)
)""")

In [8]:
#create reviews table

cursor.execute("""
CREATE TABLE Reviews(
    ReviewID SERIAL PRIMARY KEY,
    StudentUNI VARCHAR(10),
    OrderID INT,
    Rating INT,
    Comment TEXT,
    ReviewTime TIMESTAMP,
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID)
)""")

In [9]:
connection.commit()

In [34]:
#View the tables and their columns 

import pandas as pd

sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,inventory,quantity,integer,public
1,inventory,lastupdated,timestamp without time zone,public
2,inventory,fooditem,character varying,public
3,inventory,inventoryid,integer,public
4,inventory,price,numeric,public
5,inventory,expirationtime,timestamp without time zone,public
6,inventory,dininghallid,character varying,public
7,orders,paymentstatus,character varying,public
8,orders,ordertime,timestamp without time zone,public
9,orders,orderquantity,integer,public


In [35]:
with open('inventory.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'inventory', sep=',')

In [36]:
with open('orders.csv', 'r') as row:
    next(row) 
    cursor.copy_from(row, 'orders', sep=',')

In [37]:
with open('reviews.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'reviews', sep=',')

In [38]:
connection.commit()

In [40]:
def query_to_df(sql_query, connection):
    return pd.read_sql_query(sql_query, connection)

# Fetching data from each table
inventory_df = query_to_df("SELECT * FROM Inventory;", connection)
orders_df = query_to_df("SELECT * FROM Orders;", connection)
reviews_df = query_to_df("SELECT * FROM Reviews;", connection)

In [44]:
inventory_df

,inventoryid,dininghallid,fooditem,quantity,price,expirationtime,lastupdated
0,1,Chef Mike's,Italian Sub,5,5.0,2023-11-28,2023-11-23
1,2,Grace Dodge,Chocolate Cake,9,2.0,2023-11-28,2023-11-24
2,3,John Jay,Meatloaf,2,9.0,2023-11-28,2023-11-24
3,4,Faculty House,Lobster Roll,7,6.0,2023-11-29,2023-11-20
4,5,Ferris Booth Commons,Baked Ziti,10,7.0,2023-12-02,2023-11-20
5,6,Faculty House,Lobster Roll,3,4.0,2023-11-28,2023-11-21
6,7,Chef Mike's,Turkey Sub,6,8.0,2023-12-04,2023-11-25
7,8,Grace Dodge,Chocolate Cake,7,6.0,2023-12-02,2023-11-22
8,9,John Jay,Mashed Potatoes,4,9.0,2023-11-30,2023-11-21
9,10,JJ's,Chicken Wings,7,5.0,2023-11-30,2023-11-22
